From this point on, notebooks will include some default modules and configuration at the beginning. Make sure to execute this cell before any others in a notebook.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Bicycle Racing: The Effect of Air Resistance
## Lecture 3

# Realistic Projectile Motion

(The next four lectures follow closely Chapter 2 of Giordano. The text of this
chapter is available on BrightSpace/D2L.  Please read along before or after the lectures. )

In this section of the course we will study objects moving through the air under gravity and air resistance. Numerically, we will solve all our problems using the Euler method that we have already introduced.

## 2.1 Bicycle Racing and Air Resistance

![](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3b/Alexander_Vinokourov_2%2C_London_2012_Time_Trial_-_Aug_2012.jpg/640px-Alexander_Vinokourov_2%2C_London_2012_Time_Trial_-_Aug_2012.jpg)

<center> Photo by DAVID ILIFF. License: CC-BY-SA 3.0 </center>

### Objective
Understand the factors that determine the ultimate speed of a bicycle and to estimate this speed for a realistic case.

Newton's Second Law is

\begin{align}
\frac{dv}{dt} = \frac{F}{m}
\end{align}

where $F$ is the total force acting on a mass $m$ tells us how the velocity $v$ will change with time $t$.

Even if we first ignore air resistance, to accurately predict the speed of a bicycle over time would require a careful consideration of all of the forces involved and the mechanical aspects of the bike which is very complicated.

Physiological studies have shown that elite racing cyclists are able to produce approximately 400 watts over an extended period of time (~1 hr).  So, another way to approach the problem is to use work-energy ideas. That is,

$$ \frac{d E}{dt} = P $$ 
where $E$ is the energy of the bicycle-rider system and $P$ is the power output of the rider.  For a flat road, all energy is kinetic. Since $E = 1/2 mv^2$ we can derive

$$\frac{dv}{dt} = \frac{P}{mv}.$$

#### Analytical solution

Assuming $P$ is constant we can symbolically integrate this differential equation and show that

$$ v = \sqrt{v_0^2 + 2 P t /m } $$

where $v_0$ is the velocity at time $t=0$.

This analytical solution predicts that the velocity will grow forever.  This is of course not physical because we do not, yet, include the effect of air resistance.

#### Numerical solution

We can also solve this problem numerically by first discretizing the derivative

$$ \frac{dv}{dt} \approx \frac{v_{i+1} - v_i}{\Delta t} $$

where  $v_i$ is the velocity at time $t_i = i \Delta t$.

With this approximation of the derivative, we can rewrite our equation
$$ v_{i+1} = v_i + \frac{P}{m v_i} \Delta t$$

This is the same algorithm we used in Lecture 1 and 2.  We call this approach *Euler's method*. This is not the only way to discretize a derivative. In fact, later in this course we will investigate several other methods and investigate the numerical details carefully.  For now, let us continue to focus on the physics of the problem.

### No air resistance

To compute our numerical solution, let's start by assuming a value of velocity of $v_0 = 4$ m/s at $t = 0$ and take $P=400$ W as our value for a well-trained athelete. We'll also need the mass 70 kg for the bicycle-rider combination.

In [ ]:
v0 = 4.0
P = 400
m = 70

Suppose we want to compute how the velocity changes up to 100 s. We also need to choose a small enough time step.  It turns out that choosing $\Delta t =$ 1.0 s is sufficient. In general, we should come back to this step and verify that that the solution does not change much (relative error) with even smaller time steps. 

We next need to create an array for our time values

$$ [t_0, t_1, t_2, \ldots, t_{N-1} ] $$

which gives us a total of $N$ values of velocity to compute. Recall that in Python we count starting from 0.

In [ ]:
tmax = 20
dt = 1

In [ ]:
t = np.arange(0, tmax, dt)
N = len(t)
print(t)

Next, we allocate memory in the computer for velocity array of length $N$ and initialize the first element in the array to being our initial velocity.  The allocation step reserves the memory we will use to store our solution.  

In [ ]:
v = np.zeros(N)
v[0] = v0

The actual computation can now be done.  We want to loop through time while calculating all the values of velocity at each discrete time step:

In [ ]:
for i in range(N-1):
    v[i+1] = v[i] + P / (m*v[i]) * dt

You'll notice that in our loop, we go from i = 0 up to i = N-2. This is because in each iteration of the loop, we are actually calculating $v_{i+1}$.

We can print out the velocities values we have found,

In [ ]:
print(v)

It is much more descriptive to make a plot of our result.

In [ ]:
fig, axes = plt.subplots()

plt.plot(t, v)
plt.xlabel('t (s)')
plt.ylabel('v (m/s)')
plt.title('Bicyling without air resistance')

plt.show()

#### EXERCISE

> 1. Go back an rerun the simulation for 200 s.
> 2. Modify the plot so that the velocity is in km/hr and the time is measured in minutes.
> 3. Use the plot to estimate the predicted velocity at t = 3 minutes.

The plot show the velocity steadily increasing with time. After 3 minutes our model is predicting a speed faster than many cars travel.  This is clearly **unphysical** but it is happening because we have not considered air resistance.

Before proceeding, let's make copy of our result with a more descriptive name.

In [ ]:
v_no_air = v

### With air resistance

 One way of modelling air resistance is to assume a drag force of the general form

$$ F_{drag} \approx - B_1 v - B_2 v^2.$$

Unlike kinetic friction between two sliding surfaces which depends only on the normal force, drag friction depends on the velocity of the object.

At very low velocities, the behaviour is essentially linear (this is called **Stokes's Law**) and is represented by the $B_1$ term.  At more reasonable velocities the behaviour is quadratic as given by the $B_2$ term.  Unfortunately, the exact value of $B_2$ can not be calculated for complicated objects like bicyles and riders.  Approximately, however, we estimate the drag force as

 $$ F_{drag} \approx - \frac{1}{2} C \rho A v^2$$

 where $\rho$ is the density of air, $A$ is the frontal cross-sectional area of the bicycle, and $C$ is called the **drag coefficient**.  Although $C$ depends on the aerodynamics of an object, it is typically around 1.0 in value.

We can understand why the air friction has this particular mathematical form by considering the block of air that is being pushed out of the way by the bicycle and the rider.  The mass of air moved in time $dt$ is
$$m_{\mbox{air}} \sim \rho A v dt. $$

The air is given a velocity of order $v$, and hence a kinetic energy of
$$E_\mbox{air} \sim \frac{1}{2} m_\mbox{air} v^2 .$$

Because of conservation of energy, this is also the work done by the drag force (the force of the object due to air resistance, work is force times displacement) in time $dt$, 
$$ F_{\mbox{drag}} v dt = E_\mbox{air}.$$

Putting these three equations together we find

$$F_\mbox{drag} \propto \frac{1}{2} \rho A v^2.$$

The constant of proportionality, $C$, is determined by experiment.

 We can add this new force to our equation of motion as another term on the right hand side and get

$$ v_{i+1} = v_i + \frac{P}{m v_i} \Delta t - \frac{ C \rho A}{2 m} v^2 \Delta t$$

To investigate this new model numerically, we first define some additional parameters that are physically realistic.

In [ ]:
C = 0.70 # drag coefficient
rho = 1.29 # kg / m^3, density of air
A = 0.33 # m s^2, cross-sectional area

We can now compute the velocity again with this additional drag force.

In [ ]:
v = np.zeros(N)
v[0] = v0

for i in range(N-1):
    v[i+1] = v[i] + P/(m*v[i])*dt -  C*rho*A/(2*m)*v[i]**2*dt

Finally, we plot the result also with a comparison with the original computation with no air resistance.

In [ ]:
fig, axes = plt.subplots()

plt.plot(t, v_no_air, label = 'No air resistance')
plt.plot(t, v, label = 'With air resistance')
plt.xlabel('t (s)')
plt.ylabel('v (m/s)')
plt.legend()
plt.title('Bicyle simulation: velocity vs. time')

plt.show()

We can we can see that including air resistance leads to a prediction of an eventual constant speed of around 14 m/s.

In [ ]:
# we could also get this information using code
v_km_hr = v / 1000 * 60 * 60
v_eventual = v_km_hr[-1]

print('The predicted eventual constant speed, including air resistance, is {:.1f} km/hr.'.format(v_eventual))

 This model demonstrates the importance of both reducing the frontal cross-sectional areas and drag coefficient of an elite bicyclist.

### Maximum speed as function of frontal cross-sectional area

Our model suggests the imporance of minimizing the frontal cross-sectional area in order to achieve a higher maximum speed. Let's try to generalize our model answer the following

> How does maximum speed depend on cross-sectional area?


First, here is the model reorganized into separate blocks of code:

#### Initialization and constant parameters

In [ ]:
# time step
dt = 1.0
# max time
tmax = 100
# constant power
power = 400
# initial velocity
v0 = 4.00 

#### Computation routine

In [ ]:
def compute(A=0.33, C=0.7, m=70, rho=1.29):
    """Calculate the speed v of a rider of mass m
    given a drag coefficient C and air density rho"""
    
    # Initialize arrays to zero
    N = round(tmax / dt)
    t = np.zeros(N)
    v = np.zeros(N)

    # initialize velocity
    v[0] = v0

    # initialize time
    t[0] = 0

    # Calculate the solution
    for i in range(N-1):
        v[i+1] = v[i] + P/(m*v[i])*dt -  C*rho*A/(2*m)*v[i]**2*dt
        t[i+1] = t[i] + dt
        
    return t, v

#### Plotting routine

In [ ]:
def plot(t, v):
    # plot the solution
    plt.plot(t, v)
    plt.xlabel('t (s)')
    plt.ylabel('v (m/s)')
    plt.title('Bicyle simulation: velocity vs. time')

#### Main driver code 

In [ ]:
t, v = compute()
fig, axes = plt.subplots()

plot(t, v)

plt.show()

Now we can compute the solution for many possible values of cross-sectional area and plot the results.

In [ ]:
areas = np.arange(0.20, 0.50, 0.05)
max_v = np.zeros_like(areas)
N = len(areas)

fig, axes = plt.subplots()

for n in range(N):
    t, v = compute(A=areas[n])
    max_v[n] = v[-1]
    plot(t, v)

fig, axes = plt.subplots()
plt.plot(areas, max_v, 'o-')
plt.title('Maximum velocity vs cross-sectional area')
plt.xlabel('A (m$^2$)')
plt.ylabel('vmax (m/s)')

plt.show()

Our results might be useful to a performance bicycle team trying to predict the benefit in reducing the cross-sectional area by a few square centimetres.

#### Exercise
> 1. Modify the code below to investigate the impact of relative humidity on the maximum velocity.
> 2. The formulae found on [here](http://physics.holsoft.nl/physics/ocmain.htm#densair) may be useful. I've coded this up below as the function `calc_air_density()`
> 3. Considering only the impact of humidity on the drag force, what does the model predict about the change in maximum velocity?
> 4. How does your model compare with your intuition on the impact of humidity on maximum velocity? 

In [ ]:
def calc_air_density(RH=0, T=20, B=101325):
    """
    Calculate the density of humid air given
        relative humidiy, RH (0.00 - 1.00)
        temperature, T (Celsius)
        barometric pressure, B (Pa)
    """
    # temperature
    T_kelvin = T + 273.15
    # saturated vapour pressure
    p_sat = 610.7 * np.power(10, 7.5*T/(237.3+T))
    # partial vapour pressure
    p_v = RH * p_sat
    # apply formula for density of humid air
    rho_air = 1.2929 * 273.15/T_kelvin * (B - 0.3783 * p_v) / 1.013e5
    
    return rho_air

In [ ]:
# THIS IS A COPY OF THE DRIVER CODE GIVEN ABOVE. 
# IT NEEDS TO BE MODIFIED TO EXPLORE HOW 
# RELATIVE HUMIDITY AFFECTS MAXIMUM SPEED

areas = np.arange(0.20, 0.50, 0.05)
max_v = np.zeros_like(areas)
N = len(areas)

for n in range(N):
    # See the definition of compute()
    t, v = compute(A=areas[n])
    max_v[n] = v[-1]

fig, axes = plt.subplots()
plt.plot(areas, max_v, 'o-')
plt.title('Maximum velocity vs cross-sectional area')
plt.xlabel('A (m$^2$)')
plt.ylabel('vmax (m/s)')

plt.show()